# AnySat Guide

#### Simple Usage

AnySat is available through PyTorch Hub.

In [4]:
import torch

model = torch.hub.load('gastruc/anysat', 'anysat', pretrained=True, force_reload=True, flash_attn=False)

Downloading: "https://github.com/gastruc/anysat/zipball/main" to /home/GAstruc/.cache/torch/hub/main.zip


#### Local usage

Repo installation:

```bash
git clone https://github.com/gastruc/AnySat.git
cd AnySat
pip install -e AnySat
```



In [1]:
from hubconf import AnySat

model = AnySat.from_pretrained('base', flash_attn=False) #Set flash_attn=True if you have flash-attn module installed (url flash attn)
#device = "cuda" If you want to run on GPU default is cpu

#### Experiments Reproduction

All experiments are available in the [experiments](https://github.com/gastruc/AnySat/tree/main/experiments) folder.

For the reproduction of AnySat envirnoment run:

```bash
# clone project
git clone https://github.com/gastruc/anysat
cd anysat

# [OPTIONAL] create conda environment
conda create -n anysat python=3.9
conda activate anysat

# install requirements
pip install -r requirements.txt

# Create data folder where you can put your datasets
mkdir data
# Create logs folder
mkdir logs
```

And Then run the experiment you want:

```bash
# Run AnySat pretraining on GeoPlex
python src/train.py exp=GeoPlex_AnySAT

# Run AnySat finetuning on BraDD-S1TS
python src/train.py exp=BraDD_AnySAT_FT

# Run AnySat linear probing on BraDD-S1TS
python src/train.py exp=BraDD_AnySAT_LP
```

You can modify through hydra all parameters you want. For example to train a Small version of AnySat on GeoPlex datasets, run:

```bash
python src/train.py exp=GeoPlex_AnySAT model=Any_Small_multi
```



## Inference on AnySat

#### Template of data 

We are gonna use an example from S2Naip dataset.

In [2]:
import torch

In [3]:
data = {
    "s2": torch.randn(2, 4, 10, 8, 8), #2 batch size, 4 dates, 10 channels, 8x8 pixels
    "s2_dates": torch.randint(0, 367, (2, 4)),
}

You can specify the type of output you want:
- 'tile': to get the tile features
- 'patch': to get the patch features
- 'dense': to get the dense map at subpatch level
    - If dense is selected, you can specify the modality you want to keep with modality_keep parameter, default is the first modality in the data.


In [4]:
features = model(data, patch_size=10, output='tile') 
print(features.shape)

torch.Size([2, 768])


In [5]:
features = model(data, patch_size=10, output='patch') 
print(features.shape)

torch.Size([2, 64, 768])


In [6]:
features = model(data, patch_size=20, output='patch') 
print(features.shape)

torch.Size([2, 16, 768])


In [8]:
features = model(data, patch_size=80, output='patch') 
print(features.shape)

torch.Size([2, 1, 768])


In [11]:
features = model(data, patch_size=20, output='dense', modality_keep="s2") 
print(features.shape)

torch.Size([2, 16, 4, 1536])
